In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
from time import sleep
from urllib.parse import quote
from queue import Queue
import pandas as pd
import os
from selenium.webdriver.common.action_chains import ActionChains
import requests

In [103]:
# Função para acessar o portal com os parametros desejados
def consultar(navegador):
    url='https://transparencia.mprj.mp.br/web/novo-portal-transparencia/processos-distribuidos-novo'
    navegador.get(url)

In [180]:
# Iniciando drive do Chorme

# op = webdriver.ChromeOptions()
# op.add_argument('headless')
# driver = webdriver.Chrome(options=op)
driver = webdriver.Chrome()

# inicia a pagiana e espera 5 segundos para os dados serem carregados
consultar(driver)
sleep(1)

In [166]:
# Identificadores do HTML

comarca_id = 'comarcaId'
orgao_id = 'orgaoId'
tipo_documento_id = 'documentoId'
membro_id = 'membroId'

btn_buscar_id = 'export'

linha_tabela_class = 'tr-modal' #'tabela-cargos-membros'
pagination_css = 'li.page-item.active'

def get_element(id):
    element = driver.find_element(By.ID, id)
    return element


In [167]:
from datetime import datetime

def save_to_csv(name):
    file_name = f'{name}_{datetime.now().strftime("%d-%m-%Y_%H:%M:%S")}'
    scraping_values = {'Comarca': col_comarca, 'Orgão de execução': col_orgao, 'Tipo de documento': col_tipo, 'Membro': col_membro, 'Número': col_numero, 'Distribuído em': col_distribuido_em, 'Submetido em': col_submetido_em, 'Tipo': col_tipo, 'Unidade': col_unidade}
    df = pd.DataFrame.from_dict(scraping_values) 

    df.to_csv(f'{file_name}.csv')

In [172]:
comarcas_visitados = []

# manualmente parou no CAMPOS DOS GOYTACAZES

for i in range(0, 13):
    comarcas_visitados.append(comarcas.find_elements(By.TAG_NAME, "option")[i].get_attribute('innerHTML'))

comarcas_visitados

['',
 'ANGRA DOS REIS',
 'ARARUAMA',
 'ARMAÇÃO DOS BÚZIOS',
 'ARRAIAL DO CABO',
 'BARRA DO PIRAÍ',
 'BARRA MANSA',
 'BELFORD ROXO',
 'BOM JARDIM',
 'BOM JESUS DO ITABAPOANA',
 'CABO FRIO',
 'CACHOEIRAS DE MACACU',
 'CAMBUCI']

In [181]:
comarcas = get_element(comarca_id)

for comarca in comarcas.find_elements(By.TAG_NAME, "option"):
    if comarca.get_attribute('innerHTML') in comarcas_visitados:
        continue

    col_comarca = []
    col_orgao = []
    col_tipo_doc = []
    col_membro = []
    col_numero = []
    col_distribuido_em = []
    col_submetido_em = []
    col_tipo = []
    col_unidade = []   
     
    value = comarca.get_attribute('value')

    if value:
        Select(comarcas).select_by_value(value)
    else:
        continue

    name_comarca = comarca.text
    sleep(1)

    orgaos = get_element(orgao_id)
    for orgao in orgaos.find_elements(By.TAG_NAME, "option"):
        value = orgao.get_attribute('value')
        if value:
            Select(orgaos).select_by_value(value)  
        else:
            continue   

        name_orgao = orgao.text
        sleep(1)

        tipos = get_element(tipo_documento_id)
        for tipo in tipos.find_elements(By.TAG_NAME, "option"):
            value = tipo.get_attribute('value')
            if value:
                Select(tipos).select_by_value(value)  
            else:
                continue    

            name_tipo_doc = tipo.text
            sleep(1)

            membros = get_element(membro_id)
            for membro in membros.find_elements(By.TAG_NAME, "option"):
                value = membro.get_attribute('value')
                if value:
                    Select(membros).select_by_value(value)     
                    # print('membro encontrado')
                else:
                    # print('membro nao encontrado')
                    continue

                name_membro = membro.text
                                    
                # Busca
                buscar = get_element(btn_buscar_id).click() 
   
                sleep(1)          

                pages_visited = []

                actual_page = driver.find_element(By.CSS_SELECTOR, pagination_css)   
                # pages_visited.append(actual_page.text)

                # next_pages = actual_page.find_elements(By.XPATH, './/following-sibling::li')

                # for i in range(0, len(next_pages)):
                while True:
                    # print(next_pages[i].get_attribute('innerHTML'))

                    # if next_pages[i].text in pages_visited:
                    #     continue
                    linhas_tabela = driver.find_elements(By.CLASS_NAME, linha_tabela_class)  

                    for linha in linhas_tabela:     
                        # print(linha.get_attribute('innerHTML'))
                        
                        try:
                            colunas = linha.find_elements(By.TAG_NAME, 'td')
                        except Exception:
                            break

                        colunas = list(colunas)

                        # print(list(colunas)[0].text)
                        # print(colunas[1].get_attribute('text'))

                        # print(len(linhas_tabela))

                        col_numero.append(colunas[0].text)
                        col_distribuido_em.append(colunas[1].text)
                        col_submetido_em.append(colunas[2].text)
                        col_tipo.append(colunas[3].text)
                        col_unidade.append(colunas[4].text)   
                        col_comarca.append(name_comarca)
                        col_orgao.append(name_orgao)
                        col_tipo_doc.append(name_tipo_doc)
                        col_membro.append(name_membro)

                # pages_visited.append(next_pages[i].text)
                    # next_pages[i].click()         
                    next_page = actual_page.find_elements(By.XPATH, './/following-sibling::li')[-1]

                    try:
                        next_page.find_element(By.TAG_NAME, 'a') 
                    except Exception:
                        # Não tem próxima pagina
                        break

                    next_page.click()   
                        

                driver.find_element(By.CSS_SELECTOR, 'button.close.btn.btn-unstyled').click() #fechar modal

    save_to_csv(name_comarca)
    comarcas_visitados.append(comarca.get_attribute('innerHTML'))

StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7D93BEEA2+31554]
	(No symbol) [0x00007FF7D9337ED9]
	(No symbol) [0x00007FF7D91F872A]
	(No symbol) [0x00007FF7D91FE55E]
	(No symbol) [0x00007FF7D9200EB1]
	(No symbol) [0x00007FF7D9200F50]
	(No symbol) [0x00007FF7D92421DB]
	(No symbol) [0x00007FF7D926D02A]
	(No symbol) [0x00007FF7D923BA76]
	(No symbol) [0x00007FF7D926D240]
	(No symbol) [0x00007FF7D928C977]
	(No symbol) [0x00007FF7D926CDD3]
	(No symbol) [0x00007FF7D923A33B]
	(No symbol) [0x00007FF7D923AED1]
	GetHandleVerifier [0x00007FF7D96C8B1D+3217341]
	GetHandleVerifier [0x00007FF7D9715AE3+3532675]
	GetHandleVerifier [0x00007FF7D970B0E0+3489152]
	GetHandleVerifier [0x00007FF7D946E776+750614]
	(No

In [206]:
len(list(linhas_tabela))

20

In [201]:
linha.get_attribute('innerHTML')

StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7D93BEEA2+31554]
	(No symbol) [0x00007FF7D9337ED9]
	(No symbol) [0x00007FF7D91F872A]
	(No symbol) [0x00007FF7D91FE55E]
	(No symbol) [0x00007FF7D92011D8]
	(No symbol) [0x00007FF7D928D69B]
	(No symbol) [0x00007FF7D926D02A]
	(No symbol) [0x00007FF7D928C977]
	(No symbol) [0x00007FF7D926CDD3]
	(No symbol) [0x00007FF7D923A33B]
	(No symbol) [0x00007FF7D923AED1]
	GetHandleVerifier [0x00007FF7D96C8B1D+3217341]
	GetHandleVerifier [0x00007FF7D9715AE3+3532675]
	GetHandleVerifier [0x00007FF7D970B0E0+3489152]
	GetHandleVerifier [0x00007FF7D946E776+750614]
	(No symbol) [0x00007FF7D934375F]
	(No symbol) [0x00007FF7D933EB14]
	(No symbol) [0x00007FF7D933ECA2]
	(No

In [184]:
col_numero[-1]

'00270753520218190014'

In [185]:
len(col_numero)

22

In [186]:
scraping_values = {'Comarca': col_comarca, 'Orgão de execução': col_orgao, 'Tipo de documento': col_tipo, 'Membro': col_membro, 'Número': col_numero, 'Distribuído em': col_distribuido_em, 'Submetido em': col_submetido_em, 'Tipo': col_tipo, 'Unidade': col_unidade}
# scraping_values

In [194]:
col_numero

['00011445920238190014',
 '00270753520218190014',
 '00087088920238190014',
 '08132594520248190014',
 '01849252620238190001',
 '00140490420208190014',
 '00026746420248190014',
 '00026729420248190014',
 '00025343020248190014',
 '00013520920248190014',
 '00011339320248190014',
 '00010836720248190014',
 '00008610720218190014',
 '08116887320238190014',
 '00054255820238190014',
 '00022785820228190014',
 '00107926320238190014',
 '00286719820148190014',
 '08106915620248190014',
 '08140184320238190014',
 '00011445920238190014',
 '00270753520218190014']

In [187]:
df = pd.DataFrame.from_dict(scraping_values) # columns=['Número', 'Distribuído em', 'Submetido em', 'Tipo', 'Unidade'])
df

ValueError: All arrays must be of the same length

In [139]:
df.to_csv('teste.csv')